# Grad-CAM class activation visualization using `Keras Applications` 




*    on [BreaKHis](https://web.inf.ufpr.br/vri/databases/breast-cancer-histopathological-database-breakhis) dataseet 

*   the code adopted from [keras Grad-CAM doc](https://keras.io/examples/vision/grad_cam) by [fchollet](https://twitter.com/fchollet)




 

## import libraries and connect to drive

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import display
import matplotlib.cm as cm
from tensorflow.keras.preprocessing import image
import tensorflow.keras.applications as k_apps
import glob
import shutil
print(tf.version)

from google.colab import drive
drive.mount('/content/drive')

working_dir = 'drive/MyDrive/article-97/'

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.7/dist-packages/tensorflow/_api/v2/version/__init__.py'>
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Grad-CAM

### initialize model and source images 

In [ ]:
src = [
    {'fn':'1_B_TA',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_TA-14-21978AB-40-001.png'},
    {'fn':'2_B_TA',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_TA-14-21978AB-40-005.png'},
    {'fn':'3_B_TA',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_TA-14-21978AB-40-002.png'},
    {'fn':'4_B_TA',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_TA-14-21978AB-40-004.png'},
    {'fn':'5_B_TA',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_TA-14-21978AB-40-006.png'},
    {'fn':'1_B_PT',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_PT-14-22704-40-002.png'},
    {'fn':'2_B_PT',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_PT-14-22704-40-001.png'},
    {'fn':'3_B_PT',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_PT-14-22704-40-004.png'},
    {'fn':'4_B_PT',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_PT-14-22704-40-003.png'},
    {'fn':'5_B_PT',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_PT-14-22704-40-005.png'},
    {'fn':'1_B_F',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_F-14-23060AB-40-004.png'},
    {'fn':'2_B_F',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_F-14-23060AB-40-002.png'},
    {'fn':'3_B_F',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_F-14-23060AB-40-008.png'},
    {'fn':'4_B_F',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_F-14-23060AB-40-003.png'},
    {'fn':'5_B_F',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_F-14-23060AB-40-010.png'},
    {'fn':'1_B_A',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_A-14-29960CD-40-004.png'},
    {'fn':'2_B_A',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_A-14-29960CD-40-002.png'},
    {'fn':'3_B_A',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_A-14-29960CD-40-001.png'},
    {'fn':'4_B_A',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_A-14-29960CD-40-005.png'},
    {'fn':'5_B_A',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/benign/SOB_B_A-14-29960CD-40-003.png'},
    {'fn':'1_M_DC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_DC-14-13993-40-017.png'},
    {'fn':'2_M_DC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_DC-14-13993-40-014.png'},
    {'fn':'3_M_DC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_DC-14-13993-40-013.png'},
    {'fn':'4_M_DC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_DC-14-13993-40-012.png'},
    {'fn':'5_M_DC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_DC-14-13993-40-016.png'},
    {'fn':'1_M_PC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_PC-14-19440-40-001.png'},
    {'fn':'2_M_PC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_PC-14-19440-40-003.png'},
    {'fn':'3_M_PC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_PC-14-19440-40-002.png'},
    {'fn':'4_M_PC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_PC-14-19440-40-005.png'},
    {'fn':'5_M_PC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_PC-14-19440-40-004.png'},
    {'fn':'1_M_LC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_LC-14-15570C-40-003.png'},
    {'fn':'2_M_LC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_LC-14-15570C-40-002.png'},
    {'fn':'3_M_LC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_LC-14-15570C-40-001.png'},
    {'fn':'4_M_LC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_LC-14-15570C-40-008.png'},
    {'fn':'5_M_LC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_LC-14-15570C-40-006.png'},
    {'fn':'1_M_MC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_MC-14-13413-40-003.png'},
    {'fn':'2_M_MC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_MC-14-13413-40-005.png'},
    {'fn':'3_M_MC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_MC-14-13413-40-002.png'},
    {'fn':'4_M_MC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_MC-14-13413-40-004.png'},
    {'fn':'5_M_MC',  'heatmap':None,  'gradcam':None,  'path': 'drive/MyDrive/article-97/tf_files/breast_40/malignant/SOB_M_MC-14-13413-40-006.png'},
]

model_builder = k_apps.inception_resnet_v2.InceptionResNetV2
preprocess_input = k_apps.inception_resnet_v2.preprocess_input
activation = 'conv_7b_ac'

img_size = (299, 299)

model = model_builder(weights="imagenet")
model.layers[-1].activation = None

225222656/225209952 [==============================] - 1s 0us/step


### compute heatmap array and superimposed image

In [ ]:

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


def gradcam(img_path, heatmap, alpha=0.4):
    # Load the original image
    img = image.load_img(img_path)
    img = image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = image.array_to_img(superimposed_img)

    return superimposed_img

def save_res(sr):
  print('Saving  ' + sr['path'])
  display(image.load_img(sr['path']))
  plt.matshow(sr['heatmap'])
  display(sr['gradcam'])




for i in range(len(src)):
  img = image.load_img(src[i]['path'], target_size=img_size)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  src[i]['heatmap'] = make_gradcam_heatmap(img, model, activation)
  src[i]['gradcam'] = gradcam(src[i]['path'], src[i]['heatmap'])


### save results

In [ ]:
dir  = working_dir + 'grad_cam/'
for s in src:
  plt.rcParams.update({'figure.figsize': (15,15)})
  plt.matshow(s['heatmap'])
  plt.savefig(dir+s['fn']+'_hmp.png' , format="png", bbox_inches='tight')
  plt.close()

  s['gradcam'].save(dir+s['fn']+'_gdc.png')

  shutil.copyfile(s['path'], dir+s['fn']+'_org.png')
